In [2]:
### LIBS

import os
import platform
import warnings
import pandas as pd
from openpyxl import load_workbook
from openpyxl.formula.translate import Translator

warnings.filterwarnings("ignore")

In [3]:
### OPEN LOG FILE NAD GENERATE SHEETS VARIABLES

# ACTUAL
file = 'W:\\AP MM Service Request Log.xlsm'
# TESTING
# file = 'W:\\AP MM Service Request Log_CLEANOUT.xlsm'
# file = 'W:\\AP MM Service Request Log_NEW.xlsm'
# file = "W:\\VBA.xlsm"

log = load_workbook(filename=file, keep_vba=True)

ws_active=log['Active Materials']
ws_mara=log['mara']
ws_marc=log['marc']
ws_mvke=log['mvke']
ws_ausp=log['ausp']
ws_mlan=log['mlan']
ws_price=log['ZZ_MATPRC_HIST']
ws_gts=log['gts']
ws_text=log['sales text']

ws_active["A1"].value
# log.save()

'Date Added'

In [6]:
### IMPORT REQUESTS FORM DESKTOP

if platform.system() == 'Linux':
    directory = '/mnt/c/users/jzakrzewski/OneDrive - Rockwell Automation, Inc/desktop'
else:
    directory = 'c:\\users\\jzakrzewski\\OneDrive - Rockwell Automation, Inc\\desktop'

# print(directory)
requests = pd.DataFrame()

for filename in os.listdir(directory):
    if filename.endswith(".xlsm"):
        if 'AP_Material_Master_Service_Request_Form' in filename:
            file = os.path.join(directory, filename)
            print(filename)
            df = pd.read_excel(file, 2)
            df = df.iloc[1: , :]
            # print(df)
            requests = pd.concat([requests, df])

# cleanup data
requests['Unnamed: 3'] = requests['Unnamed: 3'].str.strip()
requests['Unnamed: 2'] = requests['Unnamed: 2'].str.replace('@rockwellautomation.com' , '')
requests['Unnamed: 2'] = requests['Unnamed: 2'].str.replace('@ra.rockwell.com' , '')
requests['Unnamed: 2'] = requests['Unnamed: 2'].str.lower()

requests.insert(0, 'Date', pd.to_datetime('today').strftime("%m/%d/%Y"))

requests.head(2)

2022-10-17_1335_AP_Material_Master_Service_Request_Form_FRM-10001086404-13%20(29).xlsm


,Date,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
1,10/17/2022,5008,8035,nsingh12,PN-683157,India GST review/localization,CSM India,9534-APMDPRV1,ZFG OR ZTG,Blank - No Blocks,Yes,SALES ORDER,NaN,NaN
2,10/17/2022,5008,8035,nsingh12,PN-683167,India GST review/localization,CSM India,9534-APMDPRV3,ZFG OR ZTG,Blank - No Blocks,Yes,SALES ORDER,NaN,NaN


In [7]:
### POPULATE REQUEST TO LOG 
# writer = pd.ExcelWriter("test_insert.xlsx", engine='openpyxl')
# writer = pd.ExcelWriter(file, engine='openpyxl')

# last populated row
for cell in ws_active['B']:
  # print(cell.value)
  if cell.value is None:
    # print(cell.row)
    ws_active_firstrow = cell.row 
    break


requests = requests.fillna('')
requests_output = []
for index, row in requests.iterrows():
  # print(row.values.tolist())
  ws_active.append(row.values.tolist())
  requests_output.append(row.values.tolist())
# print(requests_output)

for rowy, row in enumerate(requests_output, start = ws_active_firstrow):
  for colx, value in enumerate(row, start = 1):
    ws_active.cell(column = colx, row = rowy, value = value)

for cell in ws_active['B']:
  if cell.value is None:
    ws_active_lastrow = cell.row 
    break


### EXTEND FORMULAS By Col / Rows
column_list = ['O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'Y', 'X', 'Z', 'AA', 'AB', 'AC', 'AD', 'AE', 'AF', 'AG', 'AH', 'AI', 'AJ', 'AK', 'AL', 'AM', 'AN', 'AO', 'AP', 'AQ', 'AR', 'AS', 'AT', 'AU', 'AY', 'AZ', 'BA', 'BB', 'BC', 'BD', 'BE']
ws_active_lastrow -= 1
for x in column_list:
  i = ws_active_firstrow - 1
  # print(ws_active_firstrow, ws_active_lastrow, x, i)
  while i < ws_active_lastrow:
    i += 1
    formula = ws_active['{}2'.format(x)].value
    # print(formula)
    ws_active['{}{}'.format(x, i)] = Translator(formula, origin="{}2".format(x)).translate_formula("{}{}".format(x, i))
    # print("{}{}".format(x, i), ws_active['{}{}'.format(x, i)].value)

### CREATE SORT ORDER
sort_count = ws_active_lastrow + 1
print(sort_count)
i = 2
while i < sort_count:
  if i == 1:
    continue
  # print(i)
  ws_active["BF{}".format(i)].value = i - 1
  i += 1

# ACTUAL
file = 'W:\\AP MM Service Request Log.xlsm'
# TESTING
# file = 'test1.xlsm'
# SAVE
log.save(file)

396


In [26]:
### MAKE LIST OF MATERIALS IN AP LOG

active_matnr_list_file = r"C:\Users\jzakrzewski\OneDrive - Rockwell Automation, Inc\Desktop\ap_materials.txt"

if os.path.exists(active_matnr_list_file):
    os.remove(active_matnr_list_file)
    print('file removed')
else: print('not found')

active_matnr_list = ''
with open(active_matnr_list_file, "w") as file:
    for row in ws_active:
        if row[4].value != None: 
            if 'SAP MATNR' in row[4].value:
                continue
            else:
                active_matnr_list += (row[4].value + '\n' )
    # print(active_matnr_list)
    file.write(active_matnr_list)
    file.close()

not found
